In [1]:
import Parity_Data_Generator
import numpy as np
from pyESN import ESN
import time

N = 7000   # number of datapoints
n = 3       # n-parity
timescale = 12
rng = np.random.RandomState(42)



start_time = time.time()
# produce Data
bits, parity, target = Parity_Data_Generator.generateParityData(N, n, randomstate=rng)

# Divide in training and test data
traintest_cutoff = int(np.ceil(0.7 * len(bits)))
train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]

print("--- %s seconds ---" % (time.time() - start_time))

In [2]:
def runSomeConfigs(spectral_rad, leak_r, train_bits, test_bits, train_parity, test_parity):
#     print("Spectral-radius:",spectral_rad, "leak-rate:", leak_r)
    nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=spectral_rad,  # high spectral rad is better (high fluctuations 0->1->0)
                          leak_rate= leak_r,   # adjust leak_rate
                          sparsity=0.95,
                          noise=0.001,
                          input_shift=0,
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
                          out_activation=np.tanh,
                          inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
    # print(np.shape(bits),np.shape(parity),np.shape(target))
    nParityESN.fit(train_bits, train_parity)
    pred_test = nParityESN.predict(test_bits)

    #### not rounding
    predictedParity = pred_test # not rounding
    #### Rounding
#     better_pred_test = []           # Maybe don't round and return "true" results from pred_test
#     for x in pred_test:
#         better_pred_test.append(0 if x < 0.5 else 1)
#     predictedParity = np.array(better_pred_test).reshape(-1,1)
#     print(test_parity[40030:40100].reshape(-1))
#     print(predictedParity[40030:40100].reshape(-1))
    
    print("Testing error")
    test_error = np.sqrt(np.mean((predictedParity - test_parity) ** 2))
    print(test_error)
    return test_error

def produceTrainingData(randomState):
    bits, parity, target = Parity_Data_Generator.generateParityData(N, n, timescale=timescale, 
                                                                    randomstate=np.random.RandomState(randomState))

    # Divide in training and test data
    traintest_cutoff = int(np.ceil(0.7 * len(bits)))
    train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
    train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
#     train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]
    return train_bits, test_bits, train_parity, test_parity

In [3]:
trials = 10
# num_neurons = [500, 600, 700, 800, 900, 1000]
# num_neurons = [500]
n = 500
# num_neurons = [100, 200, 300, 400, 500, 600, 700, 800]
l_rate = np.linspace(0.1, 1, num=10)
# l_rate =[0.1]
# l_rate = [0.1,0.2,0.3]
# sp_rad = np.linspace(0.9, 1.0, num=5)
sp_rad = np.linspace(0.1, 1, num=10)
# sp_rad = [0.9,0.92,0.94,0.96,0.98, 1]

run_time = time.time()
iterations, iterationcount = trials*len(sp_rad)*len(l_rate), 0
errors = []
all_errors = [{} for _ in range(trials)] # empty dict
for i in range(trials):
    errors = []
    # produce training and testing data
    train_bits, test_bits, train_parity, test_parity = produceTrainingData(i)
    
    for l in l_rate:
        for s in sp_rad:
            print("Neurons:", n, "Spectral-radius:", s, "Leak-rate:", l)
            errors.append(runSomeConfigs(s,l, train_bits, test_bits, train_parity, test_parity))
            iterationcount = iterationcount + 1
            print("--- Progress: {:.2f}% --- {:.2f} minutes ---".format(iterationcount/iterations*100, ((time.time() - run_time)/60) ))
    ## Save leak-rate,spectral-radius and error in dictionary
    all_errors[i] = errors
            
print("Total time for Testbench:")
print("--- {:.2f} seconds ---".format(time.time() - run_time))
print("--- {:.2f} minutes ---".format(((time.time() - run_time)/60)))

Neurons: 500 Spectral-radius: 0.1 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.205471225378
Testing error
0.825548367905
--- Progress: 0.10% --- 0.21 minutes ---
Neurons: 500 Spectral-radius: 0.2 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.232380738406
Testing error
0.871539944035
--- Progress: 0.20% --- 0.38 minutes ---
Neurons: 500 Spectral-radius: 0.3 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.229515746424
Testing error
0.71083253773
--- Progress: 0.30% --- 0.56 minutes ---
Neurons: 500 Spectral-radius: 0.4 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.239108146348
Testing error
0.674206417748
--- Progress: 0.40% --- 0.74 minutes ---
Neurons: 500 Spectral-radius: 0.5 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.250179836586
Testing error
0.650526103061
--- Progress: 0.50% --- 0.91 minutes ---
Neurons: 500 Spectral-radius: 0.6 Leak-rate: 0.1
harvesting states...
fitting...
training e

In [7]:
import pickle
#### Save Data
with open('datasaveAverageSlow.p', 'wb') as outputFile:
    pickle.dump(all_errors, outputFile)
#     pickle.dump(num_neurons, outputFile)
    pickle.dump(trials, outputFile)
    pickle.dump(l_rate, outputFile)
    pickle.dump(sp_rad, outputFile)
#### Load Data
# with open('datasaveAverageSlow.p', 'rb') as inputFile:
#     all_errors = pickle.load(inputFile)
# #     num_neurons = pickle.load(inputFile)
#     trials = pickle.load(inputfile)
#     l_rate = pickle.load(inputFile)
#     sp_rad = pickle.load(inputFile)

In [20]:
### Average errors
errorLists = np.zeros(100)
for i in range(len(all_errors)):
    e = all_errors[i]
    errorLists = np.add(errorLists, np.array(e))

errorLists = np.divide(errorLists, trials)

In [21]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('HansBambel', 'T0F76G8esWCDFNmIVEUv')

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors))]
# c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]

# x, y, z = all_errors[0]
x = np.repeat(l_rate,trials)
y = np.tile(sp_rad, trials)
z = errorLists
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name=("Averaged over", trials ,"Trials"),
    mode='markers',
    marker=dict(
        size=7,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='SlowESN average Error')

In [11]:
data = [
     go.Contour(
        z=errorLists.reshape(10,10),
        x=l_rate,
        y=sp_rad,
        colorscale='Jet',
    )]
py.iplot(data)

import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('HansBambel', 'T0F76G8esWCDFNmIVEUv')

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors))]
# c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]
data = []
for i, n in enumerate(all_errors):

    x, y, z = all_errors[i]

    trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        name=(str(num_neurons[i])+" Neurons"),
        mode='markers',
        marker=dict(
            size=7,
#             color=c[i],                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8
        )
    )
    data.append(trace)

# data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-errors for different Reservoirsizes')

In [23]:
#### Get Parameters from a good performing slowESN

nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=0.8,  # high spectral rad is better (high fluctuations 0->1->0)
                          leak_rate= 0.9,   # adjust leak_rate
                          sparsity=0.95,     # 0.95
                          noise=0.01,      # 0.001
                          input_shift=-0.5,    # 0
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
                          out_activation=np.tanh,
                          inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
nParityESN.fit(train_bits, train_parity)

harvesting states...
fitting...
training error:
0.161997211128


array([[ 0.14110785],
       [ 0.07110676],
       [ 0.029205  ],
       ..., 
       [-0.01055688],
       [ 0.00571338],
       [ 0.05938461]])

In [24]:
##### Create Training Data for fastESN with output from slowESN
start_time = time.time()

new_rng = np.random.RandomState(1337)
new_N, new_n = 5000, 3
# new_bits, new_parity, new_target = Parity_Data_Generator.generateParityData(new_N, new_n, 
#                                                                             timescale=timescale,randomstate=new_rng)

# # Divide in training and test data
# traintest_cutoff = int(np.ceil(0.7 * len(bits)))
# new_train_bits, new_test_bits = new_bits[:traintest_cutoff], new_bits[traintest_cutoff:]
# # new_train_parity, new_test_parity = new_parity[:traintest_cutoff], new_parity[traintest_cutoff:]
# new_train_targets, new_test_targets = new_target[:traintest_cutoff], new_target[traintest_cutoff:]

# train_predicted_parity = nParityESN.predict(new_train_bits)
# test_predicted_parity = nParityESN.predict(new_test_bits)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0005006790161132812 seconds ---


In [27]:
#### FastESN ####
def runSomeFastConfigs(spectral_rad, leak_r, train_parity, test_parity, 
                       train_targets, test_targets, num_neurons=100):
    outputESN = ESN(n_inputs=1,
                              n_outputs=1,
                              n_reservoir=num_neurons,  
                              spectral_radius=spectral_rad,  # high spectral rad is better (high fluctuations 0->1->0)
                              leak_rate= leak_r,   # adjust leak_rate
                              sparsity=0.95,
                              noise=0.001,
                              input_shift=0,
                              input_scaling=3,  # 3 seems to be fine. Maybe use lower
                              teacher_scaling=1.12,
                              teacher_shift=-0.4,
                              out_activation=np.tanh,
                              inverse_out_activation=np.arctanh,
                              random_state=new_rng,
                              silent=False)
        # print(np.shape(bits),np.shape(parity),np.shape(target))
    outputESN.fit(train_parity, train_targets)
    pred_test = outputESN.predict(test_parity)
        
    print("Testing error")
    test_error = np.sqrt(np.mean((pred_test - test_targets) ** 2))
    print(test_error)
    return test_error

def produceFastTrainingData(randomState):
    bits, parity, target = Parity_Data_Generator.generateParityData(N, n, timescale=timescale, 
                                                                    randomstate=np.random.RandomState(randomState))

    # Divide in training and test data
    traintest_cutoff = int(np.ceil(0.7 * len(bits)))
    train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
    train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]
    
    train_predicted_parity = nParityESN.predict(train_bits)
    test_predicted_parity = nParityESN.predict(test_bits)
    
    return train_predicted_parity, test_predicted_parity, train_targets, test_targets

In [28]:
new_trials = 10
n = 100
# num_neurons_fast = [100]
# l_rate_fast = np.linspace(0.4, 1, num=7)
l_rate_fast = np.linspace(0.1, 1, num=10)
# l_rate_fast = [0.6]
# sp_rad_fast = np.linspace(0.8, 1, num=3)
sp_rad_fast = np.linspace(0.1, 1, num=10)
# sp_rad_fast = [0.95]

run_time = time.time()
iterations, iterationcount = new_trials*len(sp_rad_fast)*len(l_rate_fast), 0
errors = []
all_errors_fast = [{} for _ in range(new_trials)]
for i in range(new_trials):
    errors = []
    
    new_train_parity, new_test_parity, new_train_targets, new_test_targets = produceFastTrainingData(i)
    
    for l in l_rate_fast:
        for s in sp_rad_fast:
            print("Neurons:", n, "Spectral-radius:", s, "Leak-rate:", l)
            errors.append(runSomeFastConfigs(s,l,new_train_parity, new_test_parity, new_train_targets, new_test_targets,n))
            iterationcount = iterationcount + 1
            print("--- Progress: {:.2f}% --- {:.2f} minutes ---".format(iterationcount/iterations*100, ((time.time() - run_time)/60) ))
    ## Save leak-rate,spectral-radius and error in dictionary
    all_errors_fast[i] = errors
            
print("Total time for Testbench:")
print("--- {:.2f} seconds ---".format(time.time() - run_time))
print("--- {:.2f} minutes ---".format(((time.time() - run_time)/60)))

Neurons: 100 Spectral-radius: 0.1 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.298544456767
Testing error
0.638271950273
--- Progress: 0.10% --- 0.16 minutes ---
Neurons: 100 Spectral-radius: 0.2 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.342427533314
Testing error
0.555931656882
--- Progress: 0.20% --- 0.21 minutes ---
Neurons: 100 Spectral-radius: 0.3 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.33232248297
Testing error
0.568864261629
--- Progress: 0.30% --- 0.26 minutes ---
Neurons: 100 Spectral-radius: 0.4 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.328505705515
Testing error
0.561743814831
--- Progress: 0.40% --- 0.31 minutes ---
Neurons: 100 Spectral-radius: 0.5 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.341016385424
Testing error
0.605874405265
--- Progress: 0.50% --- 0.36 minutes ---
Neurons: 100 Spectral-radius: 0.6 Leak-rate: 0.1
harvesting states...
fitting...
training e

In [ ]:
import pickle
#### Save Data
with open('datasaveFastAverage.p', 'wb') as outputFile:
    pickle.dump(all_errors_fast, outputFile)
    pickle.dump(num_neurons_fast, outputFile)
    pickle.dump(l_rate_fast, outputFile)
    pickle.dump(sp_rad_fast, outputFile)
#### Load Data
# with open('datasaveFast.p', 'rb') as inputFile:
#     all_errors_fast = pickle.load(inputFile)
#     num_neurons_fast = pickle.load(inputFile)
#     l_rate_fast = pickle.load(inputFile)
#     sp_rad_fast = pickle.load(inputFile)

In [30]:
### Average errors
fast_errorLists = np.zeros(100)
for i in range(len(all_errors_fast)):
    e = all_errors_fast[i]
    fast_errorLists = np.add(fast_errorLists, np.array(e))

fast_errorLists = np.divide(fast_errorLists, new_trials)

In [31]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('HansBambel', 'T0F76G8esWCDFNmIVEUv')

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors_fast))]
c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]
data = []


x = np.repeat(l_rate_fast,new_trials)
y = np.tile(sp_rad_fast,new_trials)
z = fast_errorLists

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name=("Fast network averaged over",trials,"Trials"),
    mode='markers',
    marker=dict(
        size=7,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Fast Network Averaged')